In [3]:
# import libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

from tensorflow.keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

from google.cloud import storage

In [20]:
def train():
    # Read in the data
    bucket = 'rrusson-kubeflow-test'
    train_file = 'train_1247.329131066.csv'
    train_loc = "gs://{0}/{1}".format(bucket, train_file)
    train_data = pd.read_csv(train_loc, index_col=0)
    X_train = train_data.to_numpy()
    
    test_file = 'train_1247.329131066.csv'
    test_loc = "gs://{0}/{1}".format(bucket, test_file)
    test_data = pd.read_csv(test_loc, index_col=0)
    X_test = test_data.to_numpy()
    
    # reshape inputs for LSTM [samples, timesteps, features]
    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    print("Training data shape:", X_train.shape)
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    print("Test data shape:", X_test.shape)   
    
    # define the model network for the autoencoder
    def autoencoder_model(X):
        inputs = Input(shape=(X.shape[1], X.shape[2]))
        L1 = LSTM(16, activation='relu', return_sequences=True, 
                  kernel_regularizer=regularizers.l2(0.00))(inputs)
        L2 = LSTM(4, activation='relu', return_sequences=False)(L1)
        L3 = RepeatVector(X.shape[1])(L2)
        L4 = LSTM(4, activation='relu', return_sequences=True)(L3)
        L5 = LSTM(16, activation='relu', return_sequences=True)(L4)
        output = TimeDistributed(Dense(X.shape[2]))(L5)    
        model = Model(inputs=inputs, outputs=output)
        return model
    
    
    # create the autoencoder model
    model = autoencoder_model(X_train)
    model.compile(optimizer='adam', loss='mae')
    model.summary()
    
    # fit the model to the data
    nb_epochs = 10
    batch_size = 10
    model_history = model.fit(X_train, X_train, epochs=nb_epochs, batch_size=batch_size,
                        validation_split=0.1).history
    
    model.save("./model_v1.h5")
    
    return model_history

In [21]:
model_history = train()

Training data shape: (445, 1, 4)
Test data shape: (445, 1, 4)
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 1, 4)]            0         
_________________________________________________________________
lstm_24 (LSTM)               (None, 1, 16)             1344      
_________________________________________________________________
lstm_25 (LSTM)               (None, 4)                 336       
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 1, 4)              0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 1, 4)              144       
_________________________________________________________________
lstm_27 (LSTM)               (None, 1, 16)             1344      
_______________________________________________________________

In [26]:
model_history['val_loss']

[0.3058653208944533,
 0.25158346361584133,
 0.18802670968903434,
 0.14223303894201914,
 0.1321966052055359,
 0.11759434226486418,
 0.11493670112556881,
 0.11335594952106476,
 0.11371226691537434,
 0.11321881827380922]